In [1]:
from Core_Game_Parts import *
import pygame
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
import time
import os
os.environ["SDL_VIDEODRIVER"] = "windows" 

c:\Users\Yash\OneDrive\Desktop\Neural Networks Car Project\.venv\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.28.4, Python 3.13.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def model():
    """Builds a simple feedforward neural network model.

    Returns:
        model: A Keras Sequential model instance.
    """
    model=Sequential(
        [Dense(32,activation='relu',input_shape=(4,)),
         Dense(16,activation='relu'),
         Dense(3,activation='linear')]
    )
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss='mse')
    return model
ai_model=model()
ai_model.summary()
    

c:\Users\Yash\OneDrive\Desktop\Neural Networks Car Project\.venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 739 (2.89 KB)

 Trainable params: 739 (2.89 KB)

 Non-trainable params: 0 (0.00 B)

In [3]:
def run_trained_agent(weights_path):
    """
    Loads a trained agent and runs the visual simulation with corrected logic.
    """
    pygame.init()
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    clock = pygame.time.Clock()
    pygame.display.set_caption("Trained AI Agent")
    font = pygame.font.SysFont(None, 36)
    crash_font = pygame.font.SysFont(None, 72)
    track_surface = pygame.image.load(TRACK_IMAGE_PATH).convert()

    ai_model = model()
    try:
        ai_model.load_weights(weights_path)
        print(f"Successfully loaded weights from: {weights_path}")
    except Exception as e:
        print(f"Error loading weights: {e}")
        pygame.quit()
        return
    action_counts = {0: 0, 1: 0, 2: 0}  # To track the number of each action taken
    car = Car(CAR_IMAGE_PATH, DEFAULT_START_X, DEFAULT_START_Y, angle=DEFAULT_START_ANGLE)
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        # 1. Get state (Corrected to use 4 inputs)
        distances, ray_endpoints = ray_casting(car, track_surface)
        normalized_speed = car.speed / MAX_SPEED
        state = np.array(distances + [normalized_speed])
        state = np.reshape(state, [1, 4])
        # 2. Predict the best action
        q_values = ai_model.predict(state, verbose=0)
        action = np.argmax(q_values[0])
        action_map = {0: "Left", 1: "Right", 2: "Brake"}
        action_counts[action] += 1
        # 3. Execute action (using the same physics as the training function)
        car.speed += ACCELERATION
        if car.speed > 0:
            speed_factor = car.speed / MAX_SPEED
            dynamic_turn_angle = MAX_TURN_ANGLE - (speed_factor) * (MAX_TURN_ANGLE - MIN_TURN_ANGLE)
            if action == 0: car.angle += dynamic_turn_angle
            elif action == 1: car.angle -= dynamic_turn_angle
        if action == 2: car.speed -= BRAKE_FORCE
        
        car.speed -= FRICTION
        car.speed = max(0, min(car.speed, MAX_SPEED))
        car.move()

        # 4. Check for crash (Corrected to use BG_COLOR)
        crashed = False
        try:
            pixel_color = track_surface.get_at((int(car.x), int(car.y)))[:3]
            if pixel_color == DRAW_COLOR:
                crashed = True
        except IndexError:
            crashed = True
        
        if crashed:
            crash_text = crash_font.render("CRASHED! Resetting...", True, (255, 0, 0))
            text_rect = crash_text.get_rect(center=(SCREEN_WIDTH/2, SCREEN_HEIGHT/2))
            screen.blit(crash_text, text_rect)
            pygame.display.update()
            pygame.time.wait(2000)
            car = Car(CAR_IMAGE_PATH, DEFAULT_START_X, DEFAULT_START_Y, angle=DEFAULT_START_ANGLE)
            print("Total actions taken by the agent during the simulation:")
            for action, count in action_counts.items():
                print(f"Action {action}: {count}")
            action_counts = {0: 0, 1: 0, 2: 0}
            
        # 5. Render everything
        screen.blit(track_surface, (0, 0))
        car.draw(screen)
        for point in ray_endpoints:
            pygame.draw.line(screen, (0, 255, 0), (car.x, car.y), point, 1)
        
        info_text = f"Action: {action_map[action]} | Speed: {car.speed:.2f}"
        text_surface = font.render(info_text, True, (255, 255, 255))
        screen.blit(text_surface, (20, 20))

        pygame.display.update()
        clock.tick(60)

    pygame.quit()   
# --- RUN THE SIMULATION WITH YOUR TRAINED WEIGHTS ---
# ------------------------------------------------------------------
# IMPORTANT: Change this to the name of your saved weights file!
WEIGHTS_FILENAME = "dqn_car_weights_episode_50c.weights.h5"
# ------------------------------------------------------------------

# This checks if a pygame window is already open to avoid errors
if not pygame.display.get_init():
    run_trained_agent(WEIGHTS_FILENAME)
else:
    # If you get this message, restart the notebook kernel before running this cell again.
    print("Pygame window may already be open. Please restart the kernel to run again.")

c:\Users\Yash\OneDrive\Desktop\Neural Networks Car Project\.venv\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Successfully loaded weights from: dqn_car_weights_episode_50c.weights.h5
Total actions taken by the agent during the simulation:
Action 0: 38
Action 1: 79
Action 2: 6
